In [ ]:
import heapq
import time
import numpy as np

class Puzzle:
    def __init__(self, board, goal):
        self.board = board
        self.goal = goal
        self.size = len(board)
        self.goal_pos = {goal[i][j]: (i, j) for i in range(self.size) for j in range(self.size)}

    def get_neighbors(self, state):
        size = self.size
        x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
        moves = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < size and 0 <= ny < size:
                new_state = [list(row) for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                moves.append(tuple(map(tuple, new_state)))
        return moves

    def heuristic_h1(self, state):
        return sum(1 for i in range(self.size) for j in range(self.size)
                   if state[i][j] and state[i][j] != self.goal[i][j])

    def heuristic_h2(self, state):
        return sum(abs(i - self.goal_pos[val][0]) + abs(j - self.goal_pos[val][1])
                   for i, row in enumerate(state) for j, val in enumerate(row) if val)

    def solve_a_star(self, heuristic):
        start_state = tuple(map(tuple, self.board))
        pq = [(0, start_state)]
        g_score = {start_state: 0}
        came_from = {}

        while pq:
            _, current = heapq.heappop(pq)

            if current == tuple(map(tuple, self.goal)):
                return g_score[current]

            for neighbor in self.get_neighbors(current):
                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor)
                    heapq.heappush(pq, (f_score, neighbor))
                    came_from[neighbor] = current

        return -1  # No solution found


# 測試
start_board = [[5, 1, 3, 4], [2, 0, 7, 8], [9, 6, 11, 12], [13, 10, 14, 15]]
goal_board = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

puzzle = Puzzle(start_board, goal_board)

# 測試 h1
start_time = time.time()
steps_h1 = puzzle.solve_a_star(puzzle.heuristic_h1)
end_time = time.time()
print(f"h1 (錯位數) 解步驟數: {steps_h1}, 耗時: {end_time - start_time:.4f} 秒")

# 測試 h2
start_time = time.time()
steps_h2 = puzzle.solve_a_star(puzzle.heuristic_h2)
end_time = time.time()
print(f"h2 (曼哈頓距離) 解步驟數: {steps_h2}, 耗時: {end_time - start_time:.4f} 秒")


h1 (錯位數) 解步驟數: 8, 耗時: 0.0004 秒
h2 (曼哈頓距離) 解步驟數: 8, 耗時: 0.0002 秒
